In [1]:
import pandas as pd
import os
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
pd.set_option('display.max_columns', 20)

#importing the dataset
dataset_path = os.path.abspath('../Dataset/data/')
ratings = pd.read_csv(dataset_path + '/ratings.csv')
movies = pd.read_csv(dataset_path + '/movies.csv')




In [2]:

print(len(ratings))
# print(len(movies))

# Data cleaning 
movies.drop(movies[movies['genres'] == "(no genres listed)"].index, inplace = True)
ratings.drop(['timestamp'],axis = 1)
no_movies_voted = ratings.groupby('userId')['rating'].agg('count')
ratings_ = ratings.loc[no_movies_voted[no_movies_voted > 50].index,:]

print(len(ratings_))
# print(len(movies_))

# creating the final dataset containing the movies ,user and their ratings
final_dataset = ratings_.pivot(index='movieId',columns='userId',values='rating')
# final_dataset.head()
final_dataset.fillna(0,inplace=True)


from sklearn.preprocessing import StandardScaler
scaler = StandardScaler(with_mean = False)
final_dataset_scaled = scaler.fit_transform(final_dataset)

# print(final_dataset_scaled)
# Removing Sparsity
csr_data = csr_matrix(final_dataset_scaled)
final_dataset.reset_index(inplace=True)


# Using K Nearest Neighbours to find the recomendations using the similarity betwen the movies
knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=20, n_jobs=-1)
knn.fit(csr_data)




25000095
101074


NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1, n_neighbors=20)

In [6]:

def get_movie_recommendation(movie_name):
    try:
        n_movies_to_reccomend = 10
        movie_list = movies[movies['title'].str.contains(movie_name)]  
        if len(movie_list):        
            movie_idx = movie_list.iloc[0]['movieId']
            movie_idx = final_dataset[final_dataset['movieId'] == movie_idx].index[0]
            distances , indices = knn.kneighbors(csr_data[movie_idx],n_neighbors=n_movies_to_reccomend+1)    
            rec_movie_indices = sorted(list(zip(indices.squeeze().tolist(),distances.squeeze().tolist())),key=lambda x: x[1])[:0:-1]
            recommend_frame = []
            for val in rec_movie_indices:
                movie_idx = final_dataset.iloc[val[0]]['movieId']
                idx = movies[movies['movieId'] == movie_idx].index
                recommend_frame.append({'Title':movies.iloc[idx]['title'].values[0],'Distance':val[1]})
            df = pd.DataFrame(recommend_frame,index=range(n_movies_to_reccomend,0,-1))[::-1]["Title"]
            return df.values
        else:
            return "No movies found. Please check your input"
    except IndexError:
        return "No Recomendations Found"
    
    
    
print(get_movie_recommendation('Iron Man'))
print(get_movie_recommendation('Silent Voice'))


['Dark Knight, The (2008)' 'The Newcomers (2000)' 'Iron Man 2 (2010)'
 'Pagan Love Song (1950)' 'Avatar (2009)' 'Day of the Falcon (2011)'
 'Batman Begins (2005)' 'Kung Fu Panda (2008)' 'Right Cross (1950)'
 'Dangerous Method, A (2011)']
No Recomendations Found


In [ ]:
final_dataset[final_dataset['movieId'] == 166291].index